<a href="https://colab.research.google.com/github/shivicode1108/medical-agent-with-memory-using-langchain/blob/main/medical_agent_using_lang_chain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00


In [ ]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate

from langchain import OpenAI, LLMChain
from langchain.tools import DuckDuckGoSearchRun

from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
import langchain

In [ ]:
import os

In [ ]:
!pip -q install langchain tiktoken duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
#define which tools the agent can use to answer the user queries
search = DuckDuckGoSearchRun()
tools=[
    Tool(
        name="Search",
        func=search.run,
        description="Useful when needed info about current events"
    )
]

In [ ]:
obj=search.run("How can i treat a strained muscle")
type(obj)

str

In [ ]:
obj

"Treatment For immediate self-care of a muscle strain, try the R.I.C.E. approach — rest, ice, compression, elevation: Rest. Avoid activities that cause pain, swelling or discomfort. But don't avoid all physical activity. Ice. Even if you're seeking medical help, ice the area immediately. You can treat most muscle strains with at-home methods of first aid, including: Rest: Stop the physical activity that caused your strain to avoid further damaging your muscle. Ice: Apply an ice pack or cold compress for 10 to 15 minutes every hour for the first day after your injury. After one day, you can apply ice every three to four hours. Treatments Risks Prevention The RICE (rest, ice, compress, elevate) method is the most common way to treat mild muscle strains, but there are other approaches. More severe cases may need... Mild to moderate strains can be successfully treated at home with ice, heat, and anti-inflammatory medications. Severe strains or tears may require medical treatment. Symptoms 

In [ ]:
search.run("site:webmd.com how can i treat a strained muscle")

"Ice. 3 /12. To curb swelling and pain in the first 24 hours after a minor injury, apply a cold compress for 20-30 minutes. Then remove it for 20-30 minutes. You can use a bag of frozen peas. Or ... Keep in mind that half of people with the condition may have no muscle-related symptoms. Other common signs of rhabdomyolysis include: Abdominal pain. Nausea or vomiting. Fever, rapid heart rate ... Most muscle pain is not caused by myositis, but by strain injuries, or ordinary illnesses like colds and flu. These and other ordinary muscle pains are called myalgias. These and other ordinary ... Flu. The achy, sore muscle feeling you get when flu sets in is a result of your immune system's response to the virus, not the flu itself. Though the common cold and sinus troubles can also make ... 7 /12. Muscle weakness can be a sign of a serious health condition like multiple sclerosis (MS). When you have it, your immune system attacks the protective layer that surrounds your nerve fibers ..."

In [ ]:
def duck_wrapper(input_text):
  search_results=search.run(f"site:webmd.om{input_text}")
  return search_results
tools = [
    Tool(
        name = "Search WebMD",
        func=duck_wrapper,
        description="useful for when you need to answer medical and pharmalogical questions"
    )
]

Prompt Template


In [ ]:
# Set up the base template
template = """Answer the following questions as best you can, but speaking as compasionate medical professional. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin! Remember to answer as a compansionate medical professional when giving your final answer.

Question: {input}
{agent_scratchpad}"""